# A Short Tutorial on Cython: Similarity Kernels

#### Ali Caner Turkmen*
**Dept. of Computer Engineering, Bogazici University*

In this project, we had some computationally intensive requirements. Chief among them were alternating least squares, calculating a Gaussian kernel matrix, and solving the eigenvalue problem. Python, although a beautiful language, lacks in efficiency when it comes to iterative matrix operations and especially those that require slicing of large tensors.

In this light, we present a short tutorial on Cython: a surprisingly great way to do things efficiently with Python, and mix C code into Python projects. With good knowledge of Cython one can:

- Write efficient code in Python-like syntax.
- Speed things up, up to 3 orders of magnitude
- Wrap C code and call scientific C libraries.

This is why Cython is now a core part of the Python scientist's toolchain.

Before we begin, a few surprising facts about Cython:

- Python code is valid Cython code. Even without adding any syntactic Cython sugar, we can speed Python up by just compiling it to C with Cython.
- Cython has built in support for NumPy, so accessing NumPy tensors via Cython, you feel right at home.
- Cython is not a compromise! Cython feels like it is an abstraction layer over C (and due to the law of leaky abstractions), so we are trading off syntactic simplicity for little gained efficiency. This is not the case. When wrapping C or accessing Python objects, especially NumPy, Cython compiles into more efficient C code that one could write on his own. 

Another great fact is that Cython plays nicely with jupyter. We will see that we can compile a jupyter cell to C with "cell magic `%%cython`". We will use `%timeit` to check things are running faster.

## Starting Simple

Let's start simple. Say we have an array that we'd like to fill with $U(0,1)$ random numbers, but we want to do it one by one for some obscure reason. Also assume we have access to the random number generator of numpy. The classical way to do it in numpy would be:

In [7]:
import numpy as np

def fill_uni(N=10000):
    v = np.zeros(N)
    for i in range(N):
        v[i] = np.random.rand()
    return v

Let's make good on our first promise, that any Python code is valid Cython code. Let's also take advantage of the great `%%cython` cell magic.

In [8]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [11]:
%%cython
import numpy as cnp

def fill_uni_2(N=10000):
    v = cnp.zeros(N)
    for i in range(N):
        v[i] = cnp.random.rand()
    return v

In [13]:
%timeit fill_uni()
%timeit fill_uni_2()

100 loops, best of 3: 3.35 ms per loop
100 loops, best of 3: 2.34 ms per loop


We're already seeing some improvement! Now let's start shedding some of the weights Python introduces.

In [38]:
%%cython
import numpy as cnp
cimport numpy as cnp

def fill_uni_3(int N=10000):
    cdef cnp.ndarray[ndim=1, dtype=cnp.float64_t] v = cnp.zeros(N)
    cdef int i
    
    for i in range(N):
        v[i] = cnp.random.rand()
    return v

Let's look at what we did above. We typed v and i as C types. Having done that, we got rid of the thin "boxing" layer (PyObject) that Python wraps around even primitive types such as `int`.

In [19]:
%timeit fill_uni()
%timeit fill_uni_2()
%timeit fill_uni_3()

100 loops, best of 3: 3.15 ms per loop
100 loops, best of 3: 2.35 ms per loop
1000 loops, best of 3: 1.51 ms per loop


That's already a 2x improvement! We can do more. Assume we're not happy with the random number generator. We'd like to use C's good old rand().

In [40]:
%%cython
import cython
import numpy as cnp
cimport numpy as cnp

ctypedef cnp.float64_t npfloat

cdef extern from "stdlib.h":
    int rand()
    int RAND_MAX

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def fill_uni_4(int N=10000):
    cdef cnp.ndarray[ndim=1, dtype=cnp.float64_t] v = cnp.zeros(N)
    cdef int i
    
    for i in range(N):
        v[i] = <npfloat>rand() / <npfloat>RAND_MAX
    
    return v

Let's go over the code. First, we declared a `typedef` like we would do in C. We told the compiler that our `npfloat` type would refer to numpy's float64. 

Then we told the compiler which functions and constants we'd like to use from the C standard library. 

Then, we decorated our `fill_uni_4` function with `cdivision(True)`. We told the compiler to skip the Div/0 error check Python performs, and that we're sure we won't be dividing by 0. This led to a speed gain of 35% as per Cython docs.

We further tell the compiler that `boundscheck` and `wraparound` are false. In the first, we tell the compiler to stop checking for IndexErrors, and we're sure that we'll be indexing the array correctly. In the second, we tell Cython to stop checking for wraparounds in indexes (such as `A[-2]`) and that we'll index our arrays like we would index C arrays.

Finally, note the nice type casting syntax. The C casting `(float)myint` is analogous to the Cython directive `<float>myint`.

Below, we see that this led to a speed gain around 35x!!!

In [41]:
%timeit fill_uni()
%timeit fill_uni_2()
%timeit fill_uni_3()
%timeit fill_uni_4()

100 loops, best of 3: 3.26 ms per loop
100 loops, best of 3: 2.57 ms per loop
1000 loops, best of 3: 1.59 ms per loop
10000 loops, best of 3: 91.1 µs per loop


And behold, we're doing better than Numpy itself!!

In [43]:
%timeit np.random.rand(10000)

10000 loops, best of 3: 150 µs per loop


## Similarity Kernels

One of the tasks in the project required the calculation of a similarity kernel, more specifically a Gaussian (RBF) kernel defined as:

$$
K(\mathbf{x}, \mathbf{y}) = \exp\left(\dfrac{1}{2\sigma^2}||\mathbf{x} - \mathbf{y}||^2_2\right)
$$

To do this for every pair of instances, we have little choice but to launch a $O(mn^2)$ scan over all the instances. This is an ideal point in the play for Cython to take stage.

Once again, let's start naively.

In [55]:
# Assume instances are organized along the columns of X
rng = np.random.RandomState(12345)
X = rng.rand(20, 2000)

In [56]:
def rbf_naive(X, sigma=0.5):
    m, n = X.shape
    A = np.zeros((n,n))
    for i in range(n):
        for j in range(i, n):
            A[i, j] = np.exp(np.linalg.norm(X[:,i] - X[:,j])**2 / (2 * sigma**2))
    return A

In [57]:
%timeit rbf_naive(X)

1 loops, best of 3: 31.9 s per loop


We hardly want to spend 31 seconds for such a trivial task. Again, let's see what happens when we just compile the same Python code.

In [59]:
%%cython

import numpy as np

def rbf_naive_c(X, sigma=0.5):
    m, n = X.shape
    A = np.zeros((n,n))
    for i in range(n):
        for j in range(i, n):
            A[i, j] = np.exp(np.linalg.norm(X[:,i] - X[:,j])**2 / (2 * sigma**2))
    return A

In [60]:
%timeit rbf_naive_c(X)

1 loops, best of 3: 32.1 s per loop


No gain at all! This is not very surprising that although we compiled into C, we just keep working with the Python boxing layers, not shedding any of the deadweight. Let's start doing just that.

In [63]:
%%cython

import numpy as np
cimport numpy as np

ctypedef np.float64_t npfloat

def rbf_c_1(np.ndarray[ndim=2, dtype=npfloat] X, float sigma=0.5):
    cdef int m, n
    cdef np.ndarray[ndim=2, dtype=npfloat] A
    
    m = X.shape[0]
    n = X.shape[1]
    
    A = np.zeros((n,n))
    
    for i in range(n):
        for j in range(i, n):
            A[i, j] = np.exp(np.linalg.norm(X[:,i] - X[:,j])**2 / (2 * sigma**2))
    
    return A

In [64]:
%timeit rbf_c_1(X)

1 loops, best of 3: 29.9 s per loop


Still no joy.. But note the reason, although we're typing our variables we are calling Python functions with them. We must be able to do better. 

Let's work with C functions again, slice our arrays further, and use decorators to get rid some of the Python checks.

In [66]:
%%cython

import cython
import numpy as np
cimport numpy as np

cdef extern from "math.h":
    double pow(double x, double y)
    double exp(double x)

ctypedef np.float64_t npfloat

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
def rbf_c_2(np.ndarray[ndim=2, dtype=npfloat] X, float sigma=0.5):
    cdef int m, n
    cdef np.ndarray[ndim=2, dtype=npfloat] A = np.zeros((n,n))
    cdef npfloat sumsq
    
    m = X.shape[0]
    n = X.shape[1]
    
    for i in range(n):
        for j in range(i, n):
            sumsq = 0.
            for k in range(m):
                sumsq += pow(X[k, i] - X[k, j], 2)
                
            A[i, j] = exp(sumsq / (2 * pow(sigma,2)))
    
    return A

In [67]:
%timeit rbf_c_2(X)

10 loops, best of 3: 88.9 ms per loop


With adding little code, we gained 350x times speed! For greater N, the effect is bound to be much more dramatic.

## Cython For The Real World

Here we worked exclusively with the very convenient `%%cython` cell magic in Jupyter. However, when building software, we may want to (actually we must want to) have the C libraries ready for Python. 

This is also easily achieved. A Cython code file has the extension `.pyx`. A `setup.py` file is also required pointing to the compiler directives. Then, the 

```
$ python setup.py build_ext --inplace
``` 

command can be used in the command line to build the binary. Examples of both files can be found in this project's directory.